In [2]:
%load_ext autoreload
%autoreload 2

import os 
import sys

import pandas as pd

ROOT_PATH = os.path.dirname(os.getcwd())
sys.path.append(ROOT_PATH)

from src.evals.run_model import run_model, build_model
from src.config import Config
from src.utils.general import multi_target_train_test_split
from src.evals.data_processing import get_and_process_data

#### Load Data and Define Global Variables

In [3]:
DATA_PATH = os.path.join(ROOT_PATH, "data/210728_scrambles_for_unstructure_model.csv")
DATA2_PATH = os.path.join(ROOT_PATH, "data/210728_dmsv2_alldata.csv")

MODEL_TYPE = "convnet_1d"

In [4]:
X, y1, y2 = get_and_process_data(DATA_PATH)

X_train, X_test, y1_train, y1_test, y2_train, y2_test = \
        multi_target_train_test_split(X, y1, y2, return_val=False)


#### Run the model

In [7]:
model, history = run_model(data_path = DATA_PATH, model_type = MODEL_TYPE, verbose=0)

{'strides': 1, 'num_conv_layers': 2, 'kernel_size': 10, 'embdedding_output_dim': 12}
Building model: convnet_1d


KeyboardInterrupt: 

#### Evaluate

In [ ]:
loss, y1_loss, y2_loss, _, _ = model.evaluate(X_test, [y1_test, y2_test])

In [ ]:
print(f"Total loss on test set={round(loss, 3)}"
          f"\n\ttrypsin (k_T_1) loss={round(y1_loss, 3)}"
          f"\n\tthemotrypsin (k_C_1) loss={round(y2_loss,3)}")

#### Predict unfolded stability scores on the second dataset

In [9]:
def compute_stability_score(measured_EC50, predicted_EC50):
    stability_score = measured_EC50 - predicted_EC50

In [10]:
# Load all data
data2 = pd.read_csv(DATA2_PATH)

In [11]:
# Predict unfolded stability
X2, kT, kC = get_and_process_data(DATA2_PATH)
kT_pred, kC_pred = model(X2)

In [ ]:
# Get the stability scores
stability_T, stability_C = compute_stability_score(kT, kT_pred), compute_stability_score(kC, kC_pred)

Plot the correlation

In [ ]:
kT_pred, kC_pred = model(X2)